# 07 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

- 주로 SGD나 AIS 기반의 행렬 분해를 이용

In [1]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
  num_users, num_items = R.shape
  # P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력함
  np.random.seed(1)
  P = np.random.normal(scale=1./K, size=(num_users, K))
  Q = np.random.normal(scale=1./K, size=(num_items, K))

  # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
  non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

  # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
  for step in range(steps):
    for i, j, r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r - np.dot(P[i,:], Q[j,:].T)
      # Regularization을 반영한 SGD 업데이트 공식 적용
      P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 10)==0:
      print('### iteration step :', step, "rmse:", rmse)

  return P, Q

In [2]:
import warnings; warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/content/drive/MyDrive/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 칼럼을 얻기 위해 movie와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')
# column='title'로 title 칼럼으로 pivot 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [4]:
import numpy as np

def get_rmse(R, P, Q, non_zeros):
    error = 0
    for i, j, r in non_zeros:
        pred = np.dot(P[i, :], Q[j, :].T)
        error += pow(r - pred, 2)
    rmse = np.sqrt(error / len(non_zeros))
    return rmse

In [5]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01,
                            r_lambda=0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step : 0 rmse: 2.9023619751337115
### iteration step : 10 rmse: 0.7335768591017939
### iteration step : 20 rmse: 0.5115539026853438
### iteration step : 30 rmse: 0.37261628282537734
### iteration step : 40 rmse: 0.29608182991810145
### iteration step : 50 rmse: 0.2520353192341621
### iteration step : 60 rmse: 0.22487503275269882
### iteration step : 70 rmse: 0.20685455302331512
### iteration step : 80 rmse: 0.19413418783028674
### iteration step : 90 rmse: 0.1847008200272031
### iteration step : 100 rmse: 0.17742927527209082
### iteration step : 110 rmse: 0.17165226964707506
### iteration step : 120 rmse: 0.16695181946871496
### iteration step : 130 rmse: 0.16305292191997453
### iteration step : 140 rmse: 0.159766919296796
### iteration step : 150 rmse: 0.15695986999457337
### iteration step : 160 rmse: 0.15453398186715442
### iteration step : 170 rmse: 0.1524161855107769
### iteration step : 180 rmse: 0.1505508073962834
### iteration step : 190 rmse: 0.14889470913232075


In [6]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [61]:
import pandas as pd
import numpy as np

# 1) 사용자가 아직 보지 않은 영화 목록 뽑기
def get_unseen_movies(ratings_matrix: pd.DataFrame, user_id: int):
    """
    ratings_matrix: index=사용자, columns=영화, 값=평점(미시청은 0 또는 NaN)
    user_id: 추천 대상 사용자 ID (ratings_matrix의 인덱스와 동일 체계)
    return: 사용자가 아직 보지 않은 영화 리스트
    """
    if user_id not in ratings_matrix.index:
        raise KeyError(f"user_id {user_id}가 ratings_matrix.index에 없습니다.")

    # (Step 1) 해당 유저의 평점 벡터
    user_ratings = ratings_matrix.loc[user_id]

    # (Step 2) 본 영화: 평점이 0보다 크거나(명시적 평점) / NaN이 아닌 값
    seen_movies = user_ratings[user_ratings.fillna(0) > 0].index.tolist()

    # (Step 3) 전체 영화 목록에서 본 영화를 제외 → 안 본 영화
    total_movies = ratings_matrix.columns.tolist()
    unseen_list = [m for m in total_movies if m not in seen_movies]
    return unseen_list


# 2) 예측 평점 행렬에서 안 본 영화 중 top-N 추천 뽑기 (Series로 반환)
def recomm_movie_by_userid(pred_matrix: pd.DataFrame, user_id: int,
                           unseen_list: list, top_n: int = 10) -> pd.Series:
    """
    pred_matrix: index=사용자, columns=영화, 값=예측 평점
    user_id: 추천 대상 사용자
    unseen_list: get_unseen_movies로 얻은 미시청 영화 리스트
    top_n: 상위 N개
    return: index=영화, values=pred_score 인 pandas Series
    """
    if user_id not in pred_matrix.index:
        raise KeyError(f"user_id {user_id}가 pred_matrix.index에 없습니다.")

    if len(unseen_list) == 0:
        return pd.Series(dtype=float)

    # (Step 1) 해당 유저의 예측 평점에서 미시청 영화만 선택
    #         (열이 누락될 수 있으니 교집합으로 안전하게 선택)
    candidate_cols = [c for c in unseen_list if c in pred_matrix.columns]
    if len(candidate_cols) == 0:
        return pd.Series(dtype=float)

    user_pred = pred_matrix.loc[user_id, candidate_cols]

    # (Step 2) NaN 제거
    user_pred = user_pred.dropna()

    # (Step 3) 내림차순 정렬 후 top-N
    recomm_series = user_pred.sort_values(ascending=False).head(top_n)
    return recomm_series

In [62]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성.
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


# 08 파이썬 추천 시스템 패키지 - Surprise

- 추천 시스템 구축을 위한 전용 패키지
- 파이썬 기반에서 사이킷런과 유사한 API와 프레임워크를 제공
- conda나 pip를 통해 설치

In [9]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611216 sha256=d8fc9ef3e113a34d85740340a54bc2f889466a32fdbe2c2688c767809cb28608
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


### Surprise 주요 장점

- 다양한 추천 알고리즘. 예를 들어 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축 가능
- 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성됨. 예를 들어 fit(), predict() API로 추천 데이터 학습과 예측, train_test_split()으로 추천 학습 데이터 세트와 예측 데이터 세트 분리, cross_validate(), GridSearchCV 클래스를 통해 추천 시스템을 위한 모델 셀렉션, 평가, 하이퍼 파라미터 튜닝 등의 기능 제공

In [10]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [11]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [12]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [13]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25
prediction 결과의 최초 5개 추출


[Prediction(uid='917', iid='50', r_ui=3.0, est=4.149217910133892, details={'was_impossible': False}),
 Prediction(uid='235', iid='318', r_ui=5.0, est=4.333029650171127, details={'was_impossible': False}),
 Prediction(uid='339', iid='12', r_ui=5.0, est=4.652894591303144, details={'was_impossible': False}),
 Prediction(uid='878', iid='755', r_ui=2.0, est=2.7545011408291202, details={'was_impossible': False}),
 Prediction(uid='618', iid='56', r_ui=4.0, est=3.021383823173283, details={'was_impossible': False})]

In [14]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('917', '50', 4.149217910133892),
 ('235', '318', 4.333029650171127),
 ('339', '12', 4.652894591303144)]

In [15]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.37   {'was_impossible': False}


In [16]:
accuracy.rmse(predictions)

RMSE: 0.9277


0.9277328494227115

### Surprise 주요 모듈 소개

Dataset
- user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용 가능

OS 파일 데이터를 Surprise 데이터 세트로 로딩
- 주의할 점은 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안 됨

In [20]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/drive/MyDrive/ratings_noh.csv', index=False, header=False)

In [21]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/drive/MyDrive/ratings_noh.csv', reader=reader)

Surprise 데이터 세트는 기본적으로 무비렌즈 데이터 형식을 따르므로 무비렌즈 데이터 형식이 아닌 다른 OS 파일의 경우 Reader 클래스를 먼저 설정해야 함. Reader 클래스의 주요 생성 파라미터는 다음과 같음.

- line_format(string): 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식함
- sep(char): 칼럼을 분리하는 분리자이며, 디폴트는 '\t'임. 판다스 DataFrame에서 입력받을 경우 기재할 필요가 없음
- rating_scale(tuple, optional): 평점 값의 최소~최대 평점을 설정함. 디폴트는 (1,5)이지만 ratings.csv 파일의 경우는 최소 평점이 0.5, 최대 평점이 5이므로 (0.5, 5)로 설정함

In [22]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [28]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
ratings.to_csv('/content/drive/MyDrive/ratings_noh.csv', index=False, header=False)

In [29]:
from surprise import Reader
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data = Dataset.load_from_file('/content/drive/MyDrive/ratings_noh.csv', reader=reader)

In [30]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [32]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스

- SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘.
- KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘.
- BaselineOnly: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘.

### SVD 클래스의 입력 파라미터
- n_factors: 잠재 요인 K의 개수. 디폴트 100
- n_epochs: SVD 수행 시 반복 횟수. 디폴트 20
- biased: 베이스라인 사용자 편향 적용 여부이며, 디폴트는 True

### 베이스라인 평점

- 개인의 성향을 반영해 아이템 평가에 편향성(bias) 요소를 반영해 평점을 부과하는 것을 베이스라인 평점(Baseline Rating)
- 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산됨

- 전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
- 사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
- 아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

### 교차 검증과 하이퍼 파라미터 튜닝

- cross_validate()와 GridSearchCV 클래스 제공
- cross_validate: surprise.model_selection 모듈 내에 존재하며, 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차 검증 수행

In [35]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8688  0.8809  0.8679  0.8758  0.8726  0.8732  0.0048  
MAE (testset)     0.6697  0.6765  0.6686  0.6715  0.6695  0.6712  0.0028  
Fit time          1.21    1.19    1.21    1.22    1.31    1.23    0.04    
Test time         0.28    0.10    0.10    0.26    0.18    0.18    0.08    


{'test_rmse': array([0.86875695, 0.88092589, 0.86790862, 0.87578182, 0.87258328]),
 'test_mae': array([0.66974362, 0.67645239, 0.66859186, 0.67147973, 0.6695167 ]),
 'fit_time': (1.2143895626068115,
  1.1935548782348633,
  1.2122375965118408,
  1.22072434425354,
  1.3050668239593506),
 'test_time': (0.2790846824645996,
  0.0968935489654541,
  0.0965876579284668,
  0.2583601474761963,
  0.17887306213378906)}

In [37]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴트 세트로 지정, 성능 평가는 rmse, mse 로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8776388808022743
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [63]:
from surprise import Dataset, Reader, SVD

# 1) Reader 정의
reader = Reader(rating_scale=(0.5, 5.0))

# 2) pandas DataFrame -> Surprise Dataset
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

# 3) Dataset -> Trainset (내부 id 매핑 완료)
trainset = data.build_full_trainset()

# 4) 알고리즘 학습
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

In [42]:
from surprise.dataset import DatasetAutoFolds
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [43]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [46]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('/content/drive/MyDrive/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings[ 'userId' ]==9][ 'movieId' ]
if movieIds[movieIds==42].count()==0:
        print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [47]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [51]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성.
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies),
        '추천 대상 영화 수:', len(unseen_movies),
        '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [57]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
  # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98)...]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출, 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in \
                     zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
